In [102]:
import pandas as pd 

In [103]:
df = pd.read_excel('../data/processed/DOE_Stage_2.xlsx')

In [104]:
df.head()

,Month,Time Event Began,Time of Restoration,Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected
0,January,2011-01-26 09:25,2011-01-27 17:50,"Carson City, Nevada",WECC,Unknown,Cyber Event,0,0
1,February,2011-02-03 14:30,2011-02-03 18:00,"Bowie, Maryland",RFC,Unknown,Cyber Event,0,0
2,February,2011-02-17 13:00,2011-02-23 20:30,"Roseville, California",WECC,Unknown,Cyber Event,0,0
3,March,2011-03-14 07:30,2011-03-14 14:15,"Baltimore, Maryland",RFC,Unknown,Cyber Event,0,0
4,April,2011-04-03 20:23,2011-04-05 11:00,Unknown,SERC,Unknown,Cyber Event,0,0


In [105]:
col = df.columns.tolist()
df = df.rename(str.lower, axis='columns')

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3892 entries, 0 to 3891
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   month                         3892 non-null   object
 1   time event began              3892 non-null   object
 2   time of restoration           3892 non-null   object
 3   area affected                 3892 non-null   object
 4   nerc region                   3892 non-null   object
 5   alert criteria                3892 non-null   object
 6   event type                    3892 non-null   object
 7   demand loss (mw)              2431 non-null   object
 8   number of customers affected  3166 non-null   object
dtypes: object(9)
memory usage: 273.8+ KB


In [107]:
df["time event began"] = pd.to_datetime(df["time event began"], format='%Y-%m-%d %H:%M')
df["time of restoration"] = pd.to_datetime(df["time of restoration"], format='%Y-%m-%d %H:%M')

In [108]:
df['demand loss (mw)'] = pd.to_numeric(df['demand loss (mw)'], errors='coerce')
df['number of customers affected'] = pd.to_numeric(df['number of customers affected'], errors='coerce')

In [109]:
# fillna for "demand loss" based on Event type and NERC Region averages.
df['demand loss (mw)'] = df.groupby(['event type','nerc region'])['demand loss (mw)'].transform(lambda x: x.fillna(x.mean()))
df['demand loss (mw)'] = df.groupby(['event type'])['demand loss (mw)'].transform(lambda x: x.fillna(x.mean()))

df['demand loss (mw)'] = df['demand loss (mw)'].round(2)

In [110]:
# fillna for "number of customers affected" based on Event type and NERC Region averages.
df['number of customers affected'] = df.groupby(['event type','nerc region'])['number of customers affected'].transform(lambda x: x.fillna(x.mean()))
df['number of customers affected'] = df.groupby(['event type'])['number of customers affected'].transform(lambda x: x.fillna(x.mean()))


df['number of customers affected'] = df['number of customers affected'].round(0).astype('Int64')

In [111]:
# fill in remaing missing values with event type grouping
df['demand loss (mw)'] = df.groupby(['event type','nerc region'])['demand loss (mw)'].transform(lambda x: x.fillna(x.mean()))

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3892 entries, 0 to 3891
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   month                         3892 non-null   object        
 1   time event began              3892 non-null   datetime64[ns]
 2   time of restoration           3892 non-null   datetime64[ns]
 3   area affected                 3892 non-null   object        
 4   nerc region                   3892 non-null   object        
 5   alert criteria                3892 non-null   object        
 6   event type                    3892 non-null   object        
 7   demand loss (mw)              3892 non-null   float64       
 8   number of customers affected  3892 non-null   Int64         
dtypes: Int64(1), datetime64[ns](2), float64(1), object(5)
memory usage: 277.6+ KB


In [113]:
df2 = df.copy()
df2 = df2.rename(str.upper, axis='columns')

df2.to_excel('../data/processed/DOE_final.xlsx', index=False)